In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

2023-08-27 18:51:26.262339: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-27 18:51:26.343115: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-27 18:51:26.344167: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 18:51:28.258569: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load model via "saved_model" format
model = tf.keras.models.load_model('../model')

2023-08-27 18:51:39.629659: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
def preprocess_image(img_path):
    image = tf.keras.preprocessing.image.load_img(img_path) # load image
    image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY) # convert to Grayscale
    image = cv2.resize(image, (30, 30), interpolation=cv2.INTER_LINEAR) # rescale the image
    # Apply thresholding to remove noise and enhance contrast
    threshold_value = 140  # Adjust this threshold as needed
    _, image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY) # apply threshold to each pixels
    image = tf.cast(image, tf.float32) # cast pixel values to float
    image = 255 - np.array(image)
    # Define the padding values (top, bottom, left, right)
    top = 50
    bottom = 50
    left = 50
    right = 50

    # Define the padding color (white)
    padding_color = [0, 0, 0]

    # Add padding to the image
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
    image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_LINEAR) # rescale the image
    image = tf.expand_dims(image, axis=0) # add dims to the first axis
    image = tf.expand_dims(image, axis=-1) # add dims to the first axis
    
    return image

In [ ]:
preprocess_image('../')